# <center>KIỂM ĐỊNH ANOVA</center>

**ANOVA (Analysis of Variance)** có thể xem như là t-test mở rộng, nó được dùng để kiểm tra trung bình của nhiều hơn 2 nhóm có bằng nhau không?

$ H_0: \mu_1 = \mu_2 = ... = \mu_k $  
$ H_1: $ Các $\mu$ không bằng nhau  
k là số nhóm

# Điều kiện:
1. Normality: các nhóm phải có phân bố normal.
2. Homogeneity: các nhóm phải có variance như nhau.
3. Các nhóm phải độc lập với nhau

Khi các điều kiện trên không thoả, thì ta dùng **Kruskal-Wallis H-test or Welch’s ANOVA.**

Kết quả của ANOVA là dựa trên f distribution với 2 độ tự do df1=k-1, df2=n-k.

# Công thức của ANOVA là dựa trên mô hình linear regresion.

f-statistics $ = \frac{\text{(TSS-RSS)/df1}}{\text{RSS/df2}} $  
Trong đó:  
- TSS: total sum of square = $ \sum (y-\overline{y})^2 $
- RSS: residual sum of square = $ \sum (y-\hat{y})^2 $
- df1, df2: độ tự do của phân phối f, df1=k-1, df2=n-k
- n là số data, k là số nhóm

# Chỉ số eta-square với omega-square để ước tính hiệu quả của nghiên cứu với thực tế
eta-squared tương đương với $R^2$ trong linear regression, tức là giải thích variance của model so với variance của data.

$\eta^2 = R^2 = \frac{\text{TSS - RSS}}{\text{TSS}} = 1 - \frac{\text{RSS}}{\text{TSS}}$

Omega-squared là chỉ số tốt hơn eta-square vì nó không bias

<font size="+1">$ \omega^2 = \frac{\text{(TSS - RSS)} - \frac{\text{df1}}{\text{df2}} RSS   }{ \text{TSS} + \frac{1}{\text{df2}}\text{RSS} }$ </font>

Diễn giải omega-squared:
- giá trị từ -1 -> +1
- khi f-stat < 1 thì omega-squared âm
- khi omega-squared = 0 thì model không giải thích được data

# Kiểm tra điều kiện ANOVA bằng tests
Khi tính ANOVA thông qua linear regression thì các điều kiện lên data được áp dụng trên residual

1. Levene's test để kiểm tra *homogeneity of variance*
2. Shapiro's test để kiểm tra *normality*

# Post-hoc test
Kết quả anova cho phép ta nói biến số có hoặc không có liên quan outcome.  Nó không so sánh được giữa 2 nhóm trong đó với nhau. Do đó, ta cần post-hoc test.  

1. Bonferroni Correction Post-hoc Comparison  
Với f-stats, ta đặt mức có ý nghĩa là $\alpha$ là 0.05. Khi dùng Bonferroni, thì ta lấy $\alpha$ chia cho số lần so sánh là ra mức có ý nghĩa của t-test từng cặp nhóm.  
2. Tukey’s HSD Post-hoc comparison  
Tương tự như Bonferroni, tukey kiểm soát alpha chung của ANOVA = 0.05 bằng phương pháp khác, sau đó tính t-test sự khác nhau của từng cặp nhóm.

---
# Ví dụ từ https://pythonfordatascience.org/anova-python/

This is made up data that is measuring the effects of different doses of a clinical drug, Difficile, on libido. It contains 2 columns of interest, “dose” and “libido”. Dose contains information on the dosing, “placebo”, “low”, and “high”, and libido is a measure of low-high libido on a 7 point Likert scale with 7 being the highest and 1 being the lowest. 

In [1]:
import pandas as pd
import scipy.stats as st
import statsmodels.api as sm
from statsmodels.formula.api import ols
import numpy as np

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/Opensourcefordatascience/Data-sets/master/difficile.csv")
df.drop('person', axis= 1, inplace= True)

# Recoding value from numeric to string
df['dose'].replace({1: 'placebo', 2: 'low', 3: 'high'}, inplace= True)

df.head()

,dose,libido
0,placebo,3
1,placebo,2
2,placebo,1
3,placebo,1
4,placebo,4


In [21]:
# Inference
# n=15 nên dùng t statistic
def descriptive(series):
    r = st.t(len(series)-1).ppf(0.975)*series.std()/np.sqrt(len(series))
    return pd.DataFrame([[len(series),series.mean(), series.std(), np.round((series.mean()-r, series.mean()+r),3)]],
                        columns=['n','mean','std','95% conf interval'],
                        index=[series.name])

descriptive(df['libido'])

,n,mean,std,95% conf interval
libido,15,3.466667,1.76743,"[2.488, 4.445]"


In [38]:
groups = df['libido'].groupby(df['dose'])
pd.concat([descriptive(a[1]) for a in groups],
     keys=[a[0] for a in groups] )

,,n,mean,std,95% conf interval
high,libido,5,5.0,1.581139,"[3.037, 6.963]"
low,libido,5,3.2,1.303840,"[1.581, 4.819]"
placebo,libido,5,2.2,1.303840,"[0.581, 3.819]"


In [39]:
results = ols('libido ~ C(dose)', data=df).fit()
results.summary()

d:\bsthanh\Anaconda3\envs\jupyter\lib\site-packages\scipy\stats\stats.py:1450: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=15
  "anyway, n=%i" % int(n))


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 libido   R-squared:                       0.460
Model:                            OLS   Adj. R-squared:                  0.370
Method:                 Least Squares   F-statistic:                     5.119
Date:                Sat, 15 Feb 2020   Prob (F-statistic):             0.0247
Time:                        06:21:44   Log-Likelihood:                -24.683
No. Observations:                  15   AIC:                             55.37
Df Residuals:                      12   BIC:                             57.49
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              5.0000      0.627      7.972      0.000       3.634       6.366
C(dose)[T.low]        -1.8000      0.887     -2.029      0.065      -3.732       0.132
C(dose)[T.placebo]    -2.8000      0.887     -3.157      0.008      -4.732      -0.868
==============================================================================
Omnibus:                        2.517   Durbin-Watson:                   2.408
Prob(Omnibus):                  0.284   Jarque-Bera (JB):                1.108
Skew:                           0.195   Prob(JB):                        0.575
Kurtosis:                       1.727   Cond. No.                         3.73
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

---
Lưu ý Intercept trong model là high dose.  
f-stat = 5.119, p-value = 0.0247 < 0.05  
=> reject $H_0$, có sự khác nhau có ý nghĩa thống kê giữa dose của thuốc Difficile lên libido

In [47]:
aov_table = sm.stats.anova_lm(results, typ=1)
aov_table

,df,sum_sq,mean_sq,F,PR(>F)
C(dose),2.0,20.133333,10.066667,5.118644,0.024694
Residual,12.0,23.600000,1.966667,NaN,NaN


In [56]:
RSS = aov_table['sum_sq']['Residual']
TSS = sum(aov_table['sum_sq'])

In [57]:
def eta_squared(RSS,TSS):
    return 1 - RSS/TSS

eta_squared(RSS,TSS)
# tương đương R square trong ket quả linear regression

0.46036585365853644

In [61]:
def omega_squared(RSS,TSS,df1,df2):
    return (TSS-RSS-df1/df2*RSS)/(TSS+RSS/df2)

omega_squared(RSS,TSS,2,12)

0.3544857768052514